# Find the Most Underpriced Flats (Side Project)

## Import Packages

In [1]:
# Web - Scraping and API Requests
import requests
from httpx import AsyncClient, Response
from parsel import Selector
import parsel
import jmespath
import asyncio

# Data Manipulation and Analysis
import pandas as pd
from pprint import pprint 
import json
from typing import List
from typing import TypedDict

# Data Visualisation
import seaborn as sns
import matplotlib.pyplot as plt

# url displays
from IPython.display import display, Markdown

# Database Connection
from sqlalchemy import create_engine
from sqlalchemy import inspect, text

# File and System Operations
import os
import sys

## Other Setup

In [2]:
# This allows one to reload the custom package without having to install it again
%load_ext autoreload 

In [3]:
# this allows one to reload the custom package without having to install it again
%autoreload 1

sys.path.insert(0,'../src/')

# Import the custom package and sub-packages
%aimport rental_utils
%aimport rental_utils.functions
%aimport rental_utils.sql_queries

In [4]:
pd.set_option('display.max_columns', None) # Display all columns in any given DataFrame

### Import Custom Packages

In [5]:
from rental_utils import sql_queries as sqlq
from rental_utils import functions as rent

## Analyse the Data

### Get the Data from the Cloud Database

#### Finding the correct file directory for the database credentials json with the api key and password


In [ ]:
current_dir = os.path.dirname(os.path.abspath("NB05_Find_Underpriced.ipynb"))
sys.path.insert(0,os.path.join(current_dir, '..'))

credentials_file_path = os.path.join(current_dir, '..', "supabase_credentials.json")

# open the  credentials file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

#### Connect to the database

In [8]:
# connect to the database
supabase_engine = sqlq.get_supabase_engine(
    user="postgres",
    password=credentials['password'],
    host=credentials['host'],
    port=5432,
    database="postgres"
)

In [9]:
# extract the table from the database
with supabase_engine.connect() as connection:
    properties_data = pd.read_sql(text(sqlq.GET_PROPERTIES_DATA_SQL_QUERY), connection)
# check if it has loaded in correctly
properties_data.head(3)

,id,price_per_bed,predicted_price_per_bed,travel_time,distance,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription
0,50425854,2600.0,2085.82,1059,0,1,1,29,"St. George Wharf, London, SW8",51.4861,-0.12548,Apartment,new,2025-07-01T15:04:28Z,600,weekly,0,0,rent,1,52 sq. m.,/properties/50425854#/?channel=RES_LET,2017-09-07T14:00:35Z,Added today,1 bedroom apartment
1,80209919,1066.0,1918.82,1368,0,3,1,22,"Dunton Road, London, SE1",51.4926,-0.07466,Terraced,new,2025-07-01T15:46:11Z,738,weekly,0,0,rent,0,70 sq. m.,/properties/80209919#/?channel=RES_LET,2019-03-18T14:08:31Z,Added today,3 bedroom terraced house
2,82567577,2899.0,1808.56,1572,0,1,1,26,"Riverlight Quay, London, SW8",51.4803,-0.13393,Apartment,new,2025-07-01T14:57:09Z,669,weekly,0,0,rent,0,47 sq. m.,/properties/82567577#/?channel=RES_LET,2019-06-13T16:25:32Z,Added today,1 bedroom apartment


### See which Flats are Underpriced

In [16]:


def find_underpriced(df, user_budget=1200):
    """Finds underpriced flats relative to others with the same travel time
    Takes as input a dataframe with the information, and the user's budget, and outputs a sorted 
    dataframe with the most underpriced rental properties at the top, and
    a recommendation with a link to the most ideal such property."""
    # Calculate the difference between predictions and the actual price
    df = df.copy()
    df.loc[:, 'savings'] = df['predicted_price_per_bed'] - df['price_per_bed']

    # Filter out the data for only that which is within the user's input budget
    budget_data = df[df['price_per_bed'] <= user_budget]

    # sort in descending order of 'savings' column: the most undervalued flat first
    sorted_data = budget_data.sort_values(by='savings', ascending=False)

    # Output the most underpriced flat
    if not sorted_data.empty:
        top_flat = sorted_data.iloc[0]
        address = top_flat['displayAddress']
        price = top_flat['price_per_bed']
        pred_price = top_flat['predicted_price_per_bed']
        url = top_flat['propertyUrl']
        full_url = f"https://www.rightmove.co.uk{url}" if url.startswith('/') else url
        display(Markdown(
            f"Your most underpriced flat is at **{address}** for a rent per bedroom of **£{price:.2f}**, while similar properties fetch **£{pred_price:.2f}**.<br>"
            f"[Click here to view the property]({full_url})"
        ))
    else:
        print("No flats found within your budget.")
    return sorted_data

sorted_data = find_underpriced(properties_data)
sorted_data.head()


Your most underpriced flat is at **Spelman Street, London, Greater London. E1** for a rent per bedroom of **£933.33**, while similar properties fetch **£2272.29**.<br>[Click here to view the property](https://www.rightmove.co.uk/properties/164013080#/?channel=RES_LET)

,id,price_per_bed,predicted_price_per_bed,travel_time,distance,bedrooms,bathrooms,numberOfImages,displayAddress,latitude,longitude,propertySubType,listingUpdateReason,listingUpdateDate,priceAmount,priceFrequency,premiumListing,featuredProperty,transactionType,students,displaySize,propertyUrl,firstVisibleDate,addedOrReduced,propertyTypeFullDescription,savings
421,164013080,933.333,2272.29,714,0,3,1,9,"Spelman Street, London, Greater London. E1",51.5184,-0.069247,Flat,new,2025-07-01T14:23:02Z,2800,monthly,0,0,rent,0,,/properties/164013080#/?channel=RES_LET,2025-07-01T14:17:10Z,Added today,3 bedroom flat,1338.957
9,109146980,966.333,2244.18,766,0,3,1,19,"Lant Street, London, SE1",51.5013,-0.097050,Flat,new,2025-07-01T15:34:43Z,669,weekly,0,0,rent,1,57 sq. m.,/properties/109146980#/?channel=RES_LET,2021-06-19T15:53:32Z,Added today,3 bedroom flat,1277.847
282,164007377,766.667,2029.61,1163,0,3,1,7,"Braganza Street, Southwark, London, SE17",51.4876,-0.104048,Maisonette,new,2025-07-01T13:06:05Z,2300,monthly,0,0,rent,0,,/properties/164007377#/?channel=RES_LET,2025-07-01T13:00:42Z,Added today,3 bedroom maisonette,1262.943
39,156693746,875.000,2052.29,1126,0,4,2,14,"Upper North Street, London, E14",51.5116,-0.020390,Terraced,price_reduced,2025-07-01T13:13:56Z,3500,monthly,0,0,rent,0,,/properties/156693746#/?channel=RES_LET,2025-01-09T16:46:07Z,Reduced today,4 bedroom terraced house,1177.290
375,164011355,1100.000,2271.21,716,0,3,1,10,"Geoffrey House, Borough SE1",51.4973,-0.088375,Apartment,new,2025-07-01T14:01:02Z,3300,monthly,0,0,rent,0,,/properties/164011355#/?channel=RES_LET,2025-07-01T13:55:08Z,Added today,3 bedroom apartment,1171.210


In [17]:
properties_data.price_per_bed.mean()

np.float64(1733.5815187224669)